In [42]:
import requests
import time

# commander_input = "sheoldred the apocalypse"
commander_input = "zimone quandrix prodigy"
# num_ramp = 10
# num_draw = 10
# num_removal = 10
num_ramp = 5
num_draw = 5
num_removal = 20

HEADERS = {
    "User-Agent": "deck-builder-bot/1.0",
    "Accept": "application/json"
}

def get_commander(name):
    url = "https://api.scryfall.com/cards/named"
    params = {
        "exact": name
    }

    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def fetch_all_cards(query):
    url = "https://api.scryfall.com/cards/search"
    params = {"q": query}

    cards = []

    while True:
        r = requests.get(url, params=params, headers=HEADERS)
        r.raise_for_status()
        data = r.json()

        cards.extend(data["data"])

        if not data.get("has_more"):
            break

        # IMPORTANT: next_page already includes query params
        url = data["next_page"]
        params = None
        time.sleep(0.2)
    return cards


In [43]:
commander = get_commander(commander_input)

In [44]:
color_identity = commander["color_identity"]
print(commander['name'], color_identity)  # e.g. ['G', 'U', 'B', 'W']
def color_identity_query(ci):
    if not ci:
        return "ci=0"   # colorless commanders (Kozilek, etc.)
    return f"ci<={''.join(ci)}"
cid_string = color_identity_query(color_identity)
print(cid_string)

Zimone, Quandrix Prodigy ['G', 'U']
ci<=GU


In [45]:
QUERIES = {
    "ramp" : f"usd<5 function:ramp {cid_string} cmc<5 -type:land -set:UNF f:commander",
    "draw" : f"usd<5 function:draw {cid_string} cmc<5 -type:land -set:UNF f:commander",
    "removal" : f"usd<5 function:removal {cid_string} -type:land -set:UNF f:commander"
}

ramp_cards    = fetch_all_cards(QUERIES["ramp"])
draw_cards    = fetch_all_cards(QUERIES["draw"])
removal_cards = fetch_all_cards(QUERIES["removal"])

In [46]:
import random

def pick_random(cards, n):
    if len(cards) < n:
        raise ValueError(f"Not enough cards to sample {n}")
    return random.sample(cards, n)



chosen_ramp    = pick_random(ramp_cards, num_ramp)
chosen_draw    = pick_random(draw_cards, num_draw)
chosen_removal = pick_random(removal_cards, num_removal)

In [48]:
COLOR_TO_BASIC = {
    "W": "Plains",
    "U": "Island",
    "B": "Swamp",
    "R": "Mountain",
    "G": "Forest"
}

def get_basic_lands(color_identity, total_basics=16):
    if not color_identity:
        return ["Wastes"] * total_basics

    basics = []
    per_color = total_basics // len(color_identity)
    remainder = total_basics % len(color_identity)

    for c in color_identity:
        basics.extend([COLOR_TO_BASIC[c]] * per_color)

    for i in range(remainder):
        basics.append(COLOR_TO_BASIC[color_identity[i]])

    return basics

def multicolor_land_query(ci):
    return (
        f"type:land -type:basic legal:commander "
        f"ci<={''.join(ci)} "
        f"o:/add .*[{''.join(ci)}].*[{''.join(ci)}]/"
    )


FETCH_QUERY = (
    "type:land legal:commander "
    "(o:search or o:sacrifice) "
    "-o:add"
)

multicolor_lands = fetch_all_cards(multicolor_land_query(color_identity))
fetch_lands = fetch_all_cards(FETCH_QUERY)

In [49]:
import random
print('getting land base')
def pick(cards, n):
    return random.sample(cards, min(n, len(cards)))


chosen_fetches    = pick(fetch_lands, 10)

land_base = []
basic_lands = []
chosen_multicolor = []
if len(color_identity) == 1:
    print('getting only basics')
    basic_lands       = get_basic_lands(color_identity, 28)
elif len(color_identity) == 2:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 16)
    chosen_multicolor = pick(multicolor_lands, 12)
elif len(color_identity) == 3:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 8)
    chosen_multicolor = pick(multicolor_lands, 20)
elif len(color_identity) >= 4:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 4)
    chosen_multicolor = pick(multicolor_lands, 24)

# Basics (names only)
for b in basic_lands:
    land_base.append({"name": b})

# Multicolor lands
land_base.extend(chosen_multicolor)

# Fetch lands
land_base.extend(chosen_fetches)

getting land base
getting multi lands and basics


In [50]:
SOL_RING = "Sol Ring"

In [51]:
def random_filler_query(ci):
    ci_filter = f"ci<={''.join(ci)}" if ci else "ci=0"

    return (
        f"legal:commander "
        f"{ci_filter} "
        f"-type:land "
        f"-is:funny "
        f"-type:conspiracy "
        f"-type:emblem "
        f"-name:\"Sol Ring\" "
        f"usd<=1"
    )

print("fetching filler cards")
filler_cards = fetch_all_cards(random_filler_query(color_identity))
random_cards = random.sample(filler_cards, min(30, len(filler_cards)))


fetching filler cards


In [52]:
deck = []
deck.append(commander)
# Core spells
deck.extend(chosen_ramp)
deck.extend(chosen_draw)
deck.extend(chosen_removal)

# Lands
deck.extend(land_base)

# Random filler
deck.extend(random_cards)

# Sol Ring
deck.append({"name": SOL_RING})

In [53]:
from collections import Counter

counts = Counter(c["name"] for c in deck)

for name, count in sorted(counts.items()):
    print(f"{count} {name}")

1 Ancestral Statue
1 Astrolabe
1 Avacyn's Collar
1 Aven Augur
1 Band Together
1 Barrin's Unmaking
1 Beluna's Gatekeeper // Entry Denied
1 Benthic Explorers
1 Brazen Borrower // Petty Theft
1 Calamitous Tide
1 Cankerbloom
1 Child of Gaea
1 Chocobo Camp
1 Coastal Wizard
1 Containment Membrane
1 Crown of Vigor
1 Darksteel Reactor
1 Disdainful Stroke
1 Dispersal Technician
1 Dreamroot Cascade
1 Fae Offering
1 Fear of Impostors
1 Flood Plain
1 Flooded Grove
1 Forensic Researcher
8 Forest
1 Gift of Tusks
1 Glacial Chasm
1 Grasslands
1 Greater Tanuki
1 Horizon of Progress
1 Hotshot Investigators
1 Hydroid Krasis
1 Illithid Harvester // Plant Tadpoles
1 Inspiring Call
8 Island
1 Latchkey Faerie
1 Lesser Masticore
1 Lifespark Spellbomb
1 Lunatic Pandora
1 Manifold Insights
1 Meditation Pools
1 Mishra's Groundbreaker
1 Mystic Subdual
1 Nine-Ringed Bo
1 Ojer Pakpatiq, Deepest Epoch // Temple of Cyclical Time
1 Path of Ancestry
1 Perplexing Test
1 Pictures of Spider-Man
1 Prey's Vengeance
1 Quandr